In [20]:
import os
# TAVILY API KEY를 기입합니다.
# os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [21]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# TavilySearchReasults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다.
# K=5 검색결과를 5개까지 가져오겠다는 의미.

search = TavilySearchResults(k=5)

In [ ]:
#검색결과를 가져옵니다.
search.invoke("이노그리드")

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드
loader = PyPDFLoader("data/test.pdf")
# 텍스트 분활기를 사용하여 문서를 분활 합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# 문서를 로드하고 분활합니다.
split_docs = loader.load_and_split(text_splitter)
# vectorstore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# Rettriever를 생성합니다.
retriever = vector.as_retriever()

In [14]:
retriever.get_relevant_documents("19-34세 청년의 미혼 사유")

In [8]:
# langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수를 가져온다.
from langchain.tools.retriever import create_retriever_tool 

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search", 
    description="2022년 청년과 관련된 정보를 PDF 문서에서 검색합니다. '2022년 청년의 삶과 동향'에 관련된 질문은 이 도구를 사용해야 합니다. "
)

In [9]:
# tools 리스트에 search와 retriever_tool을 추가
tools = [search, retriever_tool]

In [10]:
from langchain_openai import ChatOpenAI

gpt = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

다음으로, 에이전트를 안내하기 위해 사용하고 싶은 프롬프트를 선택
이 프롬프트의 내용을 보고 싶고 LangSmith에 접근할 수 있다면, 다음으로 이동
다운로드 받아 활용할 프롬프트: https://smith.lanchain.com/hub/hwchase17/openai-functions-agent

hub.pull()함수를 사용하여 리소스를 가져온 후,
prompt.messages를 통해 해당 리소스의 메시지에 접근.

In [16]:
from langchain import hub 
# hub에서 prompt를 가져온다.
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt의 메시지를 출력합니다.
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

이제 LLM, 프롬프트 및 도구로 에이전트를 초기화할 수 있습니다. 에이전트는 입력을 받아 어떤 action을 취할지 결정하는 역할을 합니다.
중요한 것은 에이전트가 action들을 실행하지 않는 다는 것입니다. (agentexcutor에 의해 실행)

In [17]:
from langchain.agents import create_openai_functions_agent
# OpenAI 함수 기반 에이전트를 생성.
# llm, tools, prompt를 인자로 사용.
agent = create_openai_functions_agent(gpt, tools, prompt)

In [18]:
from langchain.agents import AgentExecutor
# AgentExecutor 클래스를 사용하여 agnet와 tools를 설정하고,
# 상세한 로그를 출력하도록 verbose를 true로 설정.
agent_executor = AgentExecutor(agent=agent, tools=tools, vector=True)

In [27]:
response = agent_executor.invoke(
    {"input": "2024년 19-20세 청년 동향에 대해 알려주세요."}
)
response["output"]

'2024년 19-20세 청년 동향에 대한 정보는 다음과 같습니다:\n\n- **청년 주택드림 청약통장 및 대출 출시**: 2024년 2월부터 소득 연 5천만 원 이하의 만 19~34세 무주택자를 대상으로 청약통장과 대출을 연계해 지원합니다. 이는 청년들이 보다 쉽게 자신의 집을 마련할 수 있도록 돕기 위한 조치입니다. [출처](https://korea.kr/multi/visualNewsView.do?newsId=148924647)\n\n- **고용 동향**: 2024년 2월 기준, OECD 비교기준으로 15~64세 고용률은 68.7%로 전년 동월 대비 0.7%p 상승했으며, 청년층 고용률은 46.0%로 전년 동월 대비 0.5%p 상승했습니다. 실업률은 3.2%로 전년 동월 대비 0.1%p 감소했습니다. [출처](https://korea.kr/news/policyNewsView.do?newsId=156619701)\n\n- **1월 고용 동향**: 15~64세 고용률은 68.7%로 전년 동월 대비 0.9%p 상승했고, 실업률은 3.7%로 전년 동월 대비 0.1%p 상승했습니다. 실업자 수는 1,072천 명으로 전년 동월 대비 48천 명(4.7%) 증가했으며, 청년층 실업률은 6.0%로 전년 동월 대비 0.1%p 상승했습니다. [출처](https://kostat.go.kr/board.es?mid=a10301010000&bid=210&act=view&list_no=429474)\n\n- **청년지원정책**: 2024년부터 국군장병들의 실질적인 처우 개선을 위해 병장 기준 봉급이 월 125만원으로 조정됩니다. 이는 만 19세에서 만 34세에 해당하는 청년들에게도 영향을 미칠 중요한 정책 변화 중 하나입니다. [출처](https://m.blog.naver.com/live_in_good_order/223358557737)\n\n이 정보들은 2024년 19-20세 청년들의 주거, 고용, 그리고 국군장병 처우 개선 등 다양한 분야에서의 동향과 정책 변화를 보여줍니다.'

In [28]:
response = agent_executor.invoke(
    {"input": "19-20세 청년 동향에 대해 PDF에서 알려주세요."}
)
print(f'답변: {response["output"]}')

답변: 19-20세 청년 동향에 대한 정보는 다음과 같습니다:

- **교육 및 취업 상황**: 코로나19로 인해 청년세대의 교육기회가 제한되고 있으며, 취업난으로 인해 졸업을 유예하는 사례가 증가하고 있습니다. 2020년 대학 미진학 사유로는 '돈을 벌고 싶어서'와 '가정형편이 어려워서'가 40%를 상회하며, 고등교육기관의 학사학위 취득 유예생도 전년 대비 14.1% 증가했습니다.

- **정신건강**: 청년층의 정신건강이 악화되고 있으며, 스트레스 인지율, 정신장애 유병률, 우울장애 일년유병률 등에서 일관되게 확인됩니다. 특히, 20대 초반 여성 청년의 자살률이 가파르게 증가하는 추세이며, 주요 자살충동 사유로는 경제적 어려움, 직장문제, 외로움과 고독, 가정불화 등이 있습니다.

- **여가생활**: 2020년 기준, 20대 청년은 여가생활로 취미와 오락활동에 가장 많은 시간을 할애하고 있으며, 평일 하루 평균 여가시간은 3.7시간입니다. 문화예술행사 관람률은 코로나19의 여파로 인해 전년 대비 큰 폭으로 감소했습니다.

- **문화예술행사 참여 경험**: 2020년 문화예술행사 참여 경험은 20대에서 5.7%, 30대에서 4.4%로, 대부분의 청년이 참여하지 않았습니다. 이는 코로나19로 인한 영향으로 추측됩니다.

이 정보는 청년세대의 교육, 취업, 정신건강, 여가생활 등 다양한 측면에서의 동향을 보여줍니다.
